In [36]:
%matplotlib inline
import matplotlib.pyplot as plt



import numpy as np
from gammapy.irf import *
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
from gammapy.datasets import SpectrumDatasetOnOff, SpectrumDataset, Datasets
from gammapy.data import DataStore, EventList
from gammapy.makers import SpectrumDatasetMaker
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
)
from gammapy.makers import (
    MapDatasetMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
    RingBackgroundMaker,
    ReflectedRegionsFinder,
    
)
from gammapy.maps import MapAxis, WcsNDMap, WcsGeom, Map, RegionNDMap
from gammapy.irf import load_cta_irfs
from gammapy.data import Observation
from gammapy.maps import MapAxis

from  gammapy.utils.random import get_random_state

source = "1ES1959"
data_store = DataStore.from_dir(f"/Users/cedrickperron/GAMMAPY/{source}")
hdu_table = data_store.hdu_table.read(f"/Users/cedrickperron/GAMMAPY/{source}/hdu-index.fits.gz")
obs_table = data_store.obs_table.read(f"/Users/cedrickperron/GAMMAPY/{source}/obs-index.fits.gz")
# Changing the path to the fits files in the HDU_table
hdu_table.remove_column("FILE_DIR")
hdu_table.add_column(f"/Users/cedrickperron/GAMMAPY/{source}", name="FILE_DIR")
hdu_table

OBS_ID,HDU_TYPE,HDU_CLASS,FILE_NAME,HDU_NAME,FILE_DIR
int64,bytes6,bytes10,bytes54,bytes20,str36
37793,events,events,37793.anasum.fits,EVENTS,/Users/cedrickperron/GAMMAPY/1ES1959
37793,gti,gti,37793.anasum.fits,GTI,/Users/cedrickperron/GAMMAPY/1ES1959
37793,aeff,aeff_2d,37793.anasum.fits,EFFECTIVE AREA,/Users/cedrickperron/GAMMAPY/1ES1959
37793,edisp,edisp_2d,37793.anasum.fits,ENERGY DISPERSION,/Users/cedrickperron/GAMMAPY/1ES1959
37793,psf,psf_table,37793.anasum.fits,PSF,/Users/cedrickperron/GAMMAPY/1ES1959
37890,events,events,37890.anasum.fits,EVENTS,/Users/cedrickperron/GAMMAPY/1ES1959
37890,gti,gti,37890.anasum.fits,GTI,/Users/cedrickperron/GAMMAPY/1ES1959
37890,aeff,aeff_2d,37890.anasum.fits,EFFECTIVE AREA,/Users/cedrickperron/GAMMAPY/1ES1959
37890,edisp,edisp_2d,37890.anasum.fits,ENERGY DISPERSION,/Users/cedrickperron/GAMMAPY/1ES1959


In [37]:

data_store.obs_table
data_store.hdu_table = hdu_table

data_store.hdu_table
obs_id = data_store.obs_table["OBS_ID"]



# Note NO HDU of HDU_TYPE = bkg. Write print(hdu_table) and you will see that there is no bkg.
observations = data_store.get_observations(obs_id)

No HDU found matching: OBS_ID = 37793, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 37890, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 38018, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 38019, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41713, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41960, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41961, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41962, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41963, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 41964, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 42073, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 53469, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 53470, HDU_TYPE = bkg, HDU_CLASS = None
No HDU found matching: OBS_ID = 53471, HDU_TYPE = bkg, HDU_CLASS

In [38]:
# Reconstructed and true energy axis
energy_axis = MapAxis.from_energy_bounds(0.350, 20, 20, unit= "TeV", name="energy")


energy_axis_true = MapAxis.from_energy_bounds(0.100, 50, 20, unit= "TeV", name="energy_true")


target_position = SkyCoord(ra="19h59m59.8s",dec="+65d08m55s", frame="icrs")
#print(target_position)-> In (299.99916667, 65.14861111)

on_radius=Angle("0.11 deg")
on_region= CircleSkyRegion(center=target_position,radius=on_radius)

In [39]:
geom = WcsGeom.create(skydir=target_position, binsz=0.02, width="7 deg", proj="CAR")
geom_image = geom.to_image()
# IS THERE AN EXCLUSION REGION ASIDE FROM THE CENTER???
center_region = CircleSkyRegion(center=target_position, radius=0.35 * u.deg)
exclusion_mask=Map.from_geom(geom_image)
exclusion_mask.data=geom_image.region_mask([center_region], inside=False)
#exclusion_mask.sum_over_axes().plot(add_cbar=True)


bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)

#Create a Safe_mask_maker
safe_mask_maker = SafeMaskMaker(methods=["aeff-default", "offset-max"], offset_max=2.5*u.deg)


dataset_empty = SpectrumDataset.create(
    e_reco=energy_axis, e_true=energy_axis_true, region=on_region, name="obs-0", reference_time="2010-12-02"
)
maker = SpectrumDatasetMaker(selection=["exposure", "psf", "edisp"])

In [40]:
# Define spectral model - a simple Power Law in this case
model_simu = PowerLawSpectralModel(
    index=3.0,
    amplitude=2.5e-12 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1 * u.TeV,
)
#print(model_simu)
# we set the sky model used in the dataset
model = SkyModel(spectral_model=model_simu, name="source")



In [41]:
def load_irfs_new(filename):
    from gammapy.irf import EffectiveAreaTable2D, EnergyDispersion2D, PSF3D
    aeff = EffectiveAreaTable2D.read(filename, hdu="EFFECTIVE AREA")
    #bkg = Background3D.read(filename, hdu="BACKGROUND")
    edisp = EnergyDispersion2D.read(filename, hdu="ENERGY DISPERSION")
    psf = PSF3D.read(filename, hdu="PSF")

    return dict(aeff=aeff, edisp=edisp, psf=psf)

In [42]:
list_of_dataset = Datasets()
for obs in observations:
    dataset_empty = SpectrumDataset.create(
    e_reco=energy_axis, e_true=energy_axis_true, region=on_region, name=f"obs-{obs.obs_id}", reference_time="2010-12-02")
    irfs = load_irfs_new(f"/Users/cedrickperron/GAMMAPY/{source}/{obs.obs_id}.anasum.fits")
    livetime = obs.gti.time_delta[0]
    pointing = obs.fixed_pointing_info.radec
    obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
    dataset = maker.run(dataset_empty.copy(), obs)
    dataset1 = safe_mask_maker.run(dataset_empty.copy(), obs)
    dataset.mask_safe = dataset1.mask_safe
    list_of_dataset.append(dataset)
    

In [62]:
for dataset in list_of_dataset:
    dataset.models = model
    dataset.fake(random_state=6)

In [63]:
list_of_dataset.info_table(cumulative=True)

name,counts,background,excess,sqrt_ts,npred,npred_background,npred_signal,exposure_min,exposure_max,livetime,ontime,counts_rate,background_rate,excess_rate,n_bins,n_fit_bins,stat_type,stat_sum
,,,,,,,,m2 s,m2 s,s,s,1 / s,1 / s,1 / s,,,,
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str4,float64
RZvxhn8d,10.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,3745.561543727453,322152453.28306776,1202.5660500079393,1202.5660500079393,0.008315551565699015,nan,nan,20,20,cash,24.077847547784543
RZvxhn8d,21.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,122028.0234907314,571949822.791972,2405.147428229451,1202.5813782215128,0.00873127349846456,nan,nan,20,20,cash,26.37680979423075
RZvxhn8d,31.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,125038.4430032793,867049745.9608636,3607.5989665687084,1202.5813782215128,0.00859297285736973,nan,nan,20,20,cash,30.249236944326388
RZvxhn8d,41.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,127179.27955894035,1183874180.2893608,4810.135460317135,1202.5813782215128,0.008523668478412632,nan,nan,20,20,cash,34.12166409442202
RZvxhn8d,45.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,127309.39998773007,1351011210.6901684,5411.419597089291,1202.5813782215128,0.008315747687391442,nan,nan,20,20,cash,38.559927018783256
RZvxhn8d,62.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,177187.7492337548,1575522269.2326612,6613.836187914014,1202.5813782215128,0.009374287212207871,nan,nan,20,20,cash,50.88787557633229
RZvxhn8d,79.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,231163.1344688834,1803620635.319883,7816.244543522596,1202.5813782215128,0.010107155624431957,nan,nan,20,20,cash,63.21582413388133
RZvxhn8d,96.0,nan,nan,nan,10.102710198844454,nan,10.102710198844454,302649.65602173127,2047026967.7588034,9018.592524290085,1202.5813782215128,0.010644676510380074,nan,nan,20,20,cash,73.9730154151006
